In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
#importing the data
iowa_file_path = 'train.csv'
iowa_data = pd.read_csv(iowa_file_path)


#This code predicts the sale price of the first 5 properties based on certain features.

y = iowa_data.SalePrice  #column of SalePrice
#print(iowa_data.columns)
iowa_features = ['MSSubClass','LotArea','OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd','TotalBsmtSF','1stFlrSF']
X = iowa_data[iowa_features]
iowa_model = DecisionTreeRegressor(random_state =1)
iowa_model.fit(X,y)
predicted_home_prices = iowa_model.predict(X)
print('predicted sale price based some numerical features:')


#calculating mean absolute error for for data based of features.
print(mean_absolute_error(y,predicted_home_prices))

predicted sale price based some numerical features:
33.415981735159825
